In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Sentinel 2


### Explore available images


### Download image locally


In [1]:
from spai.data.satellite import AVAILABLE_COLLECTIONS

AVAILABLE_COLLECTIONS

['sentinel-1-grd',
 'sentinel-1-rtc',
 'sentinel-2-l1c',
 'sentinel-2-l2a',
 'sentinel-3-olci',
 'sentinel-3-slstr',
 'sentinel-5p',
 'cop-dem-glo-30',
 'cop-dem-glo-90',
 'esa-worldcover',
 'landsat-8-c2-l2',
 'modis-burned-areas',
 'modis-snow-cover-8',
 'modis-snow-cover-daily',
 'alos-palsar-mosaic']

In [1]:
from spai.data.satellite import download_satellite_imagery
from spai.storage.LocalStorage import LocalStorage

storage = LocalStorage("data")

download_satellite_imagery(
    storage,
    "Barcelona",
    date=("2021-01-01", "2021-01-05"),
    collection="sentinel-3-slstr",
)

'data' folder ready to use


/Users/fran/miniconda3/envs/spai-dev/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['data/sentinel-3-slstr_2021-01-01.tif',
 'data/sentinel-3-slstr_2021-01-02.tif',
 'data/sentinel-3-slstr_2021-01-03.tif',
 'data/sentinel-3-slstr_2021-01-04.tif',
 'data/sentinel-3-slstr_2021-01-05.tif']

In [ ]:
from spai.data.satellite import explore_satellite_imagery

explore_satellite_imagery("Barcelona", collection="sentinel-2-l2a")

In [ ]:
import pystac_client
from odc.stac import load
import geopandas as gpd
from planetary_computer import sign_inplace
from planetary_computer.settings import set_subscription_key

set_subscription_key("bf441fc919b8416c93f705004ff38001")
import rioxarray


URL = "https://planetarycomputer.microsoft.com/api/stac/v1"

catalog = pystac_client.Client.open(URL, modifier=sign_inplace)
# Get collections
collections = catalog.get_collections()

bbox = (2.050623, 41.350854, 2.249584, 41.449887)

items = catalog.search(
    collections=["sentinel-5p-l2-netcdf"],
    bbox=bbox,
    datetime=("2020-01-01", "2020-01-05"),
).item_collection()

data = load(
    items,
    chunks={"time": 5, "x": 512, "y": 512},
    bands=["no2"],
    crs="epsg:4326",
    resolution=0.0045045,
    groupby="solar_day",
    bbox=bbox,
)
# select so2 variable
data.compute()

In [ ]:
longitude = 79.109
latitude = 22.746

# Get the data for the given location
data = data.sel(longitude=longitude, latitude=latitude, method="nearest")

In [ ]:
data.isel(time=0).rio.to_raster("modis-snow-day.tif")